In [ ]:
# Load a COCO-pretrained YOLOv8n model
#that will create a file called yolov8n that we will use later for prediction
model = YOLO("yolov8n.pt", "v8")

# Display model information (optional)
model.info()

In [1]:
import numpy as np
from ultralytics import YOLO
import random
import cv2
import pyttsx3  # This library will be used for voice notifications
import threading


In [2]:
# Initialize voice engine
engine = pyttsx3.init()
voices = engine.getProperty('voices')
for i, voice in enumerate(voices):
    print(f"{i}: {voice.name} ({voice.languages})")
engine.setProperty('voice', voices[1].id)
engine.setProperty('rate', 220)  # Speed of speech
engine.setProperty('volume', 1)  # Volume level (0.0 to 1.0)

0: Microsoft Hortense Desktop - French ([])
1: Microsoft Zira Desktop - English (United States) ([])


In [ ]:
my_file = open("objects.txt", "r")# this file have all objects 

# reading the file
data = my_file.read()
# split when newline ('\n') is seen
class_list = data.split("\n")
my_file.close()

In [4]:
print(class_list)


['Chair', 'Lamp', 'TrashBin', 'Tree', 'Car', 'Crosswalk', 'Person', 'Motorcycle', 'Bicycle', 'TraficLight', 'CrossSign', 'Dog', 'Wall', 'Bus', 'Cat', 'Barrier', 'DeliveryBox', 'FireHydrant', 'FallenSign', 'Fence', 'Hole_in_the_road', 'Road_cone', 'Open_manhole', 'Road_workahead', 'shopping_cart']


In [5]:
# just random colors for bounding boxes
Boxes = []
for i in range(len(class_list)):
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    Boxes.append((b, g, r))

In [6]:
#resize video frames to optimise the run
frame_wid = 1240
frame_hyt = 920

In [11]:
#cap = cv2.VideoCapture("inference/videos/afriq0.MP4") # this for a video
# Ouvrir la caméra
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

In [8]:
# Function to announce the distance and object
def announce_distance(class_name, distance):
    def speak():
        # Announce distance ranges
        if distance < 1:
            engine.say(f"Warning! A {class_name} is in front of you, less than 1 meter away.")
        elif 1 <= distance < 2:
            engine.say(f"A {class_name} is in front of you, less than 2 meters away.")
        elif 2 <= distance < 3:
            engine.say(f"A {class_name} is in front of you, about 2 to 3 meters away.")
        elif 3 <= distance < 4:
            engine.say(f"A {class_name} is in front of you, about 3 meters away.")
        elif 4 <= distance < 5:
            engine.say(f"A {class_name} is in front of you, about 4 meters away.")
        elif 5 <= distance < 6:
            engine.say(f"A {class_name} is in front of you, about 5 meters away.")
        else:
            engine.say(f"A {class_name} is far away, more than 5 meters.")

        # Run the speech engine
        engine.runAndWait()

    # Run the speaking function in a separate thread
    threading.Thread(target=speak).start()

In [ ]:
# Real widths of common objects (meters)
REAL_WIDTHS = {
    "Chair": 0.5,              # Largeur d'une chaise standard
    "Lamp": 0.3,               # Pied de lampe ou lampe de rue
    "TrashBin": 0.4,           # Poubelle de rue moyenne
    "Tree": 0.5,               # Tronc d’arbre (non la couronne)
    "Car": 1.8,                # Voiture moyenne
    "Crosswalk": 2.5,          # Largeur typique d'un passage piéton
    "Person": 0.5,             # Largeur des épaules
    "Motorcycle": 0.8,         # Largeur moyenne d'une moto
    "Bicycle": 0.6,            # Largeur guidon
    "TraficLight": 0.3,        # Colonne ou feu lui-même
    "CrossSign": 0.5,          # Panneau de signalisation piéton
    "Dog": 0.4,                # Chien moyen
    "Wall": 2.0,               # Largeur approximative d’un pan visible
    "Bus": 2.5,                # Bus standard
    "Cat": 0.25,               # Chat adulte
    "Barrier": 1.2,            # Barrière de sécurité
    "DeliveryBox": 0.5,        # Carton ou boîte de livraison
    "FireHydrant": 0.4,        # Borne incendie standard
    "FallenSign": 0.6,         # Panneau couché au sol
    "Fence": 2.0,              # Largeur visible d'une barrière ou clôture
    "Hole_in_the_road": 0.7,   # Diamètre moyen visible
    "Road_cone": 0.3,          # Cône de signalisation
    "Open_manhole": 0.6,       # Bouche d’égout ouverte
    "Road_workahead": 1.0,     # Panneau ou zone de travaux
    "shopping_cart": 0.55      # Largeur moyenne d’un chariot
}



FOCAL_LENGTH = 462  # You need to calculate this once using a known object

# Load YOLO model
model = YOLO("runs/detect/train/weights/best.pt", "v8")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame. Exiting ...")
        break

    overlay = frame.copy()
    cv2.putText(overlay, "Montrez un objet pour détecter...", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Perform object detection using YOLO model
    detect_params = model.predict(source=[frame], conf=0.45, save=False)
    DP = detect_params[0].cpu().numpy()

    if len(DP) != 0:
        if len(detect_params[0].boxes) != 0:
            for i in range(len(detect_params[0].boxes)):
                boxes = detect_params[0].boxes
                box = boxes[i]
                clsID = int(box.cls.cpu().numpy()[0])
                conf = round(float(box.conf.cpu().numpy()[0]), 3)
                bb = box.xyxy.cpu().numpy()[0]

            class_name = class_list[clsID]

            # ===================== CALCUL DISTANCE =====================
            x1, y1, x2, y2 = map(int, bb)
            object_roi = frame[y1:y2, x1:x2]

            gray = cv2.cvtColor(object_roi, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            distance_text = "Distance: ? m"

            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                rect = cv2.minAreaRect(largest_contour)
                object_width_pixels = min(rect[1])  

                object_width_real = REAL_WIDTHS.get(class_name, None)

                if object_width_real and object_width_pixels > 0:
                    # Calculate the distance based on object width in pixels
                    distance = (object_width_real * FOCAL_LENGTH) / object_width_pixels
                    distance_text = f"Distance: {round(distance, 2)} m"
                    #if(class_name!= "person"):
                        # Announce the distance for this object
                    announce_distance(class_name, distance)
                    

            # ===================== DESSINER =====================
            cv2.rectangle(frame, (x1, y1), (x2, y2), Boxes[clsID], 3)

            # Line 1: Object name + confidence
            cv2.putText(frame, f"{class_name} {conf}%", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

            # Line 2: Distance
            cv2.putText(frame, distance_text, (x1, y1 + 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Display frame
    cv2.imshow("ObjectDetection", frame if len(DP) != 0 else overlay)

    # Quit if 'q' is pressed or window is closed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    if cv2.getWindowProperty("ObjectDetection", cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)